In [1]:
from dataclasses import dataclass
from dataset_creation.generate_img_dataset import *

In [2]:
@dataclass
class Config:
    ckpt = "stable_diffusion/models/ldm/stable-diffusion-v1/v1-5-pruned-emaonly.ckpt"
    vae_ckpt = "stable_diffusion/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt"
    steps = 100
    max_p2p = 0.9
    min_p2p = 0.1
    max_cfg = 15
    min_cfg = 7.5
opt = Config()

In [3]:
global_seed = torch.randint(1 << 32, ()).item()
print(f"Global seed: {global_seed}")
seed_everything(global_seed)
model = load_model_from_config(
    OmegaConf.load("stable_diffusion/configs/stable-diffusion/v1-inference.yaml"),
    ckpt=opt.ckpt,
    vae_ckpt=opt.vae_ckpt,
)


Seed set to 2196380451


Global seed: 2196380451
Loading model from stable_diffusion/models/ldm/stable-diffusion-v1/v1-5-pruned-emaonly.ckpt
Global Step: 840000
Loading VAE from stable_diffusion/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [4]:
model.half().cuda().eval()
model_wrap = k_diffusion.external.CompVisDenoiser(model)

In [5]:
with torch.no_grad(), torch.autocast("cuda"), model.ema_scope():
    uncond = model.get_learned_conditioning(2 * [""])
    sigmas = model_wrap.get_sigmas(opt.steps)


In [6]:
uncond.shape

torch.Size([2, 77, 768])

In [7]:
prompt = {
    "caption": "A painting of a cat",
    "output": "A painting of a dog"
    }
cond = model.get_learned_conditioning([prompt["caption"], prompt["output"]])

In [8]:
cond.shape

torch.Size([2, 77, 768])

In [9]:
cond[0]

tensor([[-0.3884,  0.0229, -0.0523,  ..., -0.4902, -0.3066,  0.0674],
        [ 0.0292, -1.3271,  0.3074,  ..., -0.5264,  0.9756,  0.6655],
        [-0.1973,  0.2935,  1.3066,  ..., -1.9863, -1.2363,  1.7627],
        ...,
        [-0.7173, -0.4338,  0.7124,  ..., -0.2522, -0.2900, -1.4990],
        [-0.7124, -0.4385,  0.7188,  ..., -0.2449, -0.2749, -1.4912],
        [-0.7251, -0.3877,  0.7461,  ..., -0.2423, -0.2440, -1.4932]],
       device='cuda:0', dtype=torch.float16)

In [10]:
cond[1]

tensor([[-0.3884,  0.0229, -0.0523,  ..., -0.4902, -0.3066,  0.0674],
        [ 0.0292, -1.3271,  0.3074,  ..., -0.5264,  0.9756,  0.6655],
        [-0.1973,  0.2935,  1.3066,  ..., -1.9863, -1.2363,  1.7627],
        ...,
        [-0.8286, -0.0626,  0.6226,  ..., -0.3594, -1.2246, -0.6938],
        [-0.8291, -0.0596,  0.6323,  ..., -0.3560, -1.2070, -0.6802],
        [-0.8174, -0.0274,  0.6577,  ..., -0.3765, -1.1562, -0.6841]],
       device='cuda:0', dtype=torch.float16)

In [11]:
print(sigmas[0])
x = torch.randn(1, 4, 512 // 8, 512 // 8, device="cuda") * sigmas[0]
x = repeat(x, "1 ... -> n ...", n=2)
x.shape

tensor(14.6090, device='cuda:0')


torch.Size([2, 4, 64, 64])

In [12]:
model_wrap_cfg = CFGDenoiser(model_wrap)
p2p_threshold = opt.min_p2p + torch.rand(()).item() * (opt.max_p2p - opt.min_p2p)
cfg_scale = opt.min_cfg + torch.rand(()).item() * (opt.max_cfg - opt.min_cfg)
print(f"p2p_threshold: {p2p_threshold} | cfg_scale: {cfg_scale}")

p2p_threshold: 0.7287426948547363 | cfg_scale: 8.345257490873337


In [13]:
seed = 0
torch.manual_seed(seed)
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    extra_args = {"cond": cond, "uncond": uncond, "cfg_scale": cfg_scale}
    samples_ddim = sample_euler_ancestral(model_wrap_cfg, x, sigmas, p2p_threshold, **extra_args)
    x_samples_ddim = model.decode_first_stage(samples_ddim)
    x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

RuntimeError: "upsample_nearest2d_nhwc_out_frame" not implemented for 'BFloat16'